In [90]:
import numpy as np
import matplotlib.pyplot as plt
import addict
import itertools

In [109]:
cfg = addict.Dict(system_size=3,
                  hidden_size=10)

In [312]:
arange = np.arange(cfg.system_size)
index_list = list(itertools.product(arange, repeat=2))

In [313]:
index_arr = np.empty(len(index_list), dtype='O')

In [314]:
index_arr[:] = index_list

In [315]:
index_arr = index_arr.reshape((cfg.system_size, cfg.system_size))

In [316]:
index_arr

array([[(0, 0), (0, 1), (0, 2)],
       [(1, 0), (1, 1), (1, 2)],
       [(2, 0), (2, 1), (2, 2)]], dtype=object)

In [317]:
arange = np.arange(cfg.system_size ** 2)
grid = arange.reshape((cfg.system_size, cfg.system_size))
# four nearest-neighbors
self_and_neighbors = np.zeros((cfg.system_size, cfg.system_size, 5), dtype=int)
self_and_neighbors[..., 0] = grid
neighbor_index = 1
for axis in [0, 1]:
  for shift in [-1, 1]:
    self_and_neighbors[..., neighbor_index] = np.roll(grid, shift=shift, axis=axis)
    neighbor_index += 1
# reshape to (num_latent, num_neighbors)
self_and_neighbors = self_and_neighbors.reshape(cfg.system_size ** 2, -1)

In [207]:
d = np.arange(cfg.system_size ** 2)

In [321]:
m = np.random.randint(cfg.system_size ** 2 - 1, size=cfg.hidden_size)

In [322]:
mask = np.zeros((cfg.hidden_size, cfg.system_size ** 2), dtype=bool)

In [323]:
m

array([2, 2, 5, 6, 3, 3, 5, 1, 7, 5])

In [324]:
for i in range(len(mask)):
  row = self_and_neighbors[d[m[i]]]
  true_values = (m[i] >= row)
  mask[i, row[true_values]] = True

In [325]:
mask

array([[ True, False, False, False, False, False, False, False, False],
       [ True, False, False, False, False, False, False, False, False],
       [ True, False, False,  True, False, False, False, False, False],
       [False,  True, False, False,  True, False,  True, False, False],
       [False,  True, False,  True, False, False, False, False, False],
       [False,  True, False,  True, False, False, False, False, False],
       [ True, False, False,  True, False, False, False, False, False],
       [ True,  True, False, False, False, False, False, False, False],
       [False, False,  True, False, False,  True,  True,  True, False],
       [ True, False, False,  True, False, False, False, False, False]])

In [369]:
mh = np.random.randint(cfg.system_size ** 2 - 1, size=cfg.hidden_size)

In [370]:
maskh = np.zeros((cfg.hidden_size, cfg.hidden_size), dtype=bool)
for i in range(len(mask)):
  row = self_and_neighbors[m[mh[i]]]
  true_values = (mh[i] >= row)
  maskh[i, row[true_values]] = True

In [371]:
print(maskh)

[[False False  True False False False False False False False]
 [False False  True False False False False False False False]
 [False False  True False False False False False False False]
 [ True False False False False False False False False False]
 [ True False False False False False False False False False]
 [ True  True  True False  True False False  True False False]
 [ True False False  True  True False False False False False]
 [ True False False  True  True False False False False False]
 [ True False False False False False False False False False]
 [ True False False  True False False False False False False]]


In [372]:
mh

array([2, 2, 2, 0, 0, 7, 4, 4, 0, 3])

In [373]:
self_and_neighbors[mh[0]]

array([2, 5, 8, 0, 1])

In [374]:
maskh = np.zeros((cfg.hidden_size, cfg.hidden_size), dtype=bool)
for i in range(len(maskh)):
  maskh[i] = np.logical_and(np.isin(m, self_and_neighbors[mh[i]]), mh[i] >= m)

In [375]:
print(maskh)

[[ True  True False False False False False  True False False]
 [ True  True False False False False False  True False False]
 [ True  True False False False False False  True False False]
 [False False False False False False False False False False]
 [False False False False False False False False False False]
 [False False False  True False False False  True  True False]
 [False False False False  True  True False  True False False]
 [False False False False  True  True False  True False False]
 [False False False False False False False False False False]
 [False False False False  True  True False False False False]]


In [376]:
mo = np.arange(cfg.system_size ** 2)

In [377]:
masko = np.zeros((cfg.system_size ** 2, cfg.hidden_size), dtype=bool)
for i in range(len(masko)):
  masko[i] = np.logical_and(np.isin(mh, self_and_neighbors[mo[i]]), mo[i] > mh)

In [378]:
print(masko)

[[False False False False False False False False False False]
 [False False False  True  True False False False  True False]
 [False False False  True  True False False False  True False]
 [False False False  True  True False False False  True False]
 [False False False False False False False False False  True]
 [ True  True  True False False False  True  True False  True]
 [False False False  True  True False False False  True  True]
 [False False False False False False  True  True False False]
 [ True  True  True False False  True False False False False]]


In [379]:
mh

array([2, 2, 2, 0, 0, 7, 4, 4, 0, 3])

In [380]:
np.isin(mh, self_and_neighbors[mo[8]]), mo[8] > mh

(array([ True,  True,  True, False, False,  True, False, False, False,
        False]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]))

In [382]:
mask.shape, maskh.shape, masko.shape

((10, 9), (10, 10), (9, 10))